# Rakentaminen Mistral-mallien avulla

## Johdanto

Tässä osiossa käydään läpi:
- Eri Mistral-mallien tarkastelu
- Mallien käyttötarkoitukset ja tilanteet, joissa niitä kannattaa hyödyntää
- Koodiesimerkit, jotka havainnollistavat kunkin mallin erityispiirteitä


## Mistral-mallit

Tässä osiossa tutustumme kolmeen eri Mistral-malliin: **Mistral Large**, **Mistral Small** ja **Mistral Nemo**.

Kaikki nämä mallit ovat ilmaiseksi saatavilla Github Model -markkinapaikalla. Tämän muistion koodi käyttää näitä malleja koodin suorittamiseen. Lisätietoja Github-mallien käytöstä [AI-mallien prototyyppien tekemiseen](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst) löydät täältä.


## Mistral Large 2 (2407)
Mistral Large 2 on tällä hetkellä Mistralin lippulaivamalli ja suunniteltu yrityskäyttöön.

Malli on parannus alkuperäiseen Mistral Largeen tarjoamalla
- Suurempi kontekstin ikkuna – 128k vs 32k
- Parempi suorituskyky matemaattisissa ja koodaus-tehtävissä – 76,9 % keskimääräinen tarkkuus vs 60,4 %
- Parantunut monikielinen suorituskyky – tuetut kielet: englanti, ranska, saksa, espanja, italia, portugali, hollanti, venäjä, kiina, japani, korea, arabia ja hindi.

Näiden ominaisuuksien ansiosta Mistral Large on erinomainen
- *Retrieval Augmented Generation (RAG)* – suuremman kontekstin ikkunan ansiosta
- *Function Calling* – tässä mallissa on sisäänrakennettu funktiokutsujen tuki, mikä mahdollistaa integraation ulkoisiin työkaluihin ja API-rajapintoihin. Kutsut voidaan tehdä rinnakkain tai peräkkäin järjestyksessä.
- *Koodin generointi* – tämä malli on erityisen hyvä Python-, Java-, TypeScript- ja C++-koodin tuottamisessa.


### RAG-esimerkki käyttäen Mistral Large 2


Tässä esimerkissä käytämme Mistral Large 2 -mallia RAG-mallin toteuttamiseen tekstidokumentilla. Kysymys on kirjoitettu koreaksi ja koskee kirjoittajan toimia ennen yliopistoa.

Käytössä on Cohere Embeddings Model, jolla luodaan upotuksia sekä tekstidokumentista että kysymyksestä. Tässä esimerkissä vektorivarastona toimii faiss Python -paketti.

Mistral-mallille lähetettävä kehotus sisältää sekä kysymykset että haetut tekstipätkät, jotka ovat samankaltaisia kysymyksen kanssa. Malli tuottaa tämän jälkeen luonnollisen kielen vastauksen.


In [50]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [51]:
import requests
import numpy as np
import faiss
import os

from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
from azure.ai.inference import EmbeddingsClient

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = requests.get('https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt')
text = response.text

chunk_size = 2048
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
len(chunks)

embed_model_name = "cohere-embed-v3-multilingual" 

embed_client = EmbeddingsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(token)
)

embed_response = embed_client.embed(
    input=chunks,
    model=embed_model_name
)



text_embeddings = []
for item in embed_response.data:
    length = len(item.embedding)
    text_embeddings.append(item.embedding)
text_embeddings = np.array(text_embeddings)


d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

question = "저자가 대학에 오기 전에 주로 했던 두 가지 일은 무엇이었나요?？"

question_embedding = embed_client.embed(
    input=[question],
    model=embed_model_name
)

question_embeddings = np.array(question_embedding.data[0].embedding)


D, I = index.search(question_embeddings.reshape(1, -1), k=2) # distance, index
retrieved_chunks = [chunks[i] for i in I.tolist()[0]]

prompt = f"""
Context information is below.
---------------------
{retrieved_chunks}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""


chat_response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful assistant."),
        UserMessage(content=prompt),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(chat_response.choices[0].message.content)

The author primarily engaged in two activities before college: writing and programming. In terms of writing, they wrote short stories, albeit not very good ones, with minimal plot and characters expressing strong feelings. For programming, they started writing programs on the IBM 1401 used for data processing during their 9th grade, at the age of 13 or 14. They used an early version of Fortran and typed programs on punch cards, later loading them into the card reader to run the program.


## Mistral Small
Mistral Small on toinen malli Mistral-malliperheessä, joka kuuluu premier/enterprise-kategoriaan. Kuten nimestä voi päätellä, kyseessä on Small Language Model (SLM). Mistral Smallin etuja ovat:
- Säästää kustannuksissa verrattuna Mistral LLM -malleihin, kuten Mistral Large ja NeMo – hinnassa jopa 80 % pudotus
- Matala viive – nopeampi vasteaika verrattuna Mistralin LLM-malleihin
- Joustava – voidaan ottaa käyttöön erilaisissa ympäristöissä, ja vaatimukset resursseille ovat vähäisemmät

Mistral Small sopii erinomaisesti:
- Tekstipohjaisiin tehtäviin, kuten tiivistämiseen, tunteiden analysointiin ja kääntämiseen
- Sovelluksiin, joissa tehdään usein pyyntöjä, sillä se on kustannustehokas
- Matalaa viivettä vaativiin kooditehtäviin, kuten koodin tarkasteluun ja ehdotuksiin


## Mistral Smallin ja Mistral Largen vertailu

Näyttääksesi viive-eroja Mistral Smallin ja Largen välillä, suorita alla olevat solut.

Huomaat eron vasteajoissa, joka on noin 3-5 sekuntia. Kiinnitä myös huomiota vastausten pituuteen ja tyyliin samalla kehotteella.


In [ ]:
import os 
endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-small"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

## Mistral NeMo

Verrattuna kahteen muuhun tässä oppitunnissa käsiteltyyn malliin, Mistral NeMo on ainoa ilmainen malli, jolla on Apache2-lisenssi.

Sitä pidetään parannuksena aiempaan Mistralin avoimen lähdekoodin LLM:ään, Mistral 7B:hen.

Muita NeMo-mallin ominaisuuksia ovat:

- *Tehokkaampi tokenisointi:* Tämä malli käyttää Tekken-tokenisoijaa yleisemmin käytetyn tiktokenin sijaan. Tämän ansiosta suorituskyky on parempi useammilla kielillä ja koodilla.

- *Jälkikoulutus:* Perusmalli on saatavilla jälkikoulutukseen. Tämä mahdollistaa joustavuuden käyttötapauksissa, joissa jälkikoulutusta saatetaan tarvita.

- *Luontainen funktiokutsu* – Kuten Mistral Large, myös tämä malli on koulutettu funktiokutsuihin. Tämä tekee siitä ainutlaatuisen, sillä se on yksi ensimmäisistä avoimen lähdekoodin malleista, jotka tukevat tätä ominaisuutta.


## Mistral NeMo

Verrattuna kahteen muuhun tässä oppitunnissa käsiteltyyn malliin, Mistral NeMo on ainoa ilmainen malli, jolla on Apache2-lisenssi.

Sitä pidetään parannuksena aiempaan Mistralin avoimen lähdekoodin LLM:ään, Mistral 7B:hen.

Muita NeMo-mallin ominaisuuksia ovat:

- *Tehokkaampi tokenisointi:* Tämä malli käyttää Tekken-tokenisoijaa yleisemmin käytetyn tiktokenin sijaan. Tämä mahdollistaa paremman suorituskyvyn useammilla kielillä ja koodilla.

- *Hienosäätö:* Perusmalli on saatavilla hienosäätöä varten. Tämä tuo lisää joustavuutta käyttötapauksiin, joissa hienosäätöä saatetaan tarvita.

- *Natiivi funktiokutsu* – Kuten Mistral Large, myös tämä malli on koulutettu funktiokutsuihin. Tämä tekee siitä ainutlaatuisen, sillä se on yksi ensimmäisistä avoimen lähdekoodin malleista, jotka tukevat tätä ominaisuutta.


### Tokenisaattoreiden vertailu

Tässä esimerkissä tarkastelemme, miten Mistral NeMo käsittelee tokenisointia verrattuna Mistral Largeen.

Molemmissa esimerkeissä käytetään samaa kehotetta, mutta huomaat, että NeMo palauttaa vähemmän tokeneita kuin Mistral Large.


In [11]:
pip install mistral-common

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.0/797.0 kB 15.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "open-mistral-nemo	"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

128


In [13]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "mistral-large-latest"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

135


## Oppiminen ei lopu tähän – jatka matkaa

Kun olet suorittanut tämän oppitunnin, tutustu [Generative AI Learning -kokoelmaamme](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst) ja jatka Generatiivisen tekoälyn osaamisesi kehittämistä!



---

**Vastuuvapauslauseke**:  
Tämä asiakirja on käännetty käyttämällä tekoälypohjaista käännöspalvelua [Co-op Translator](https://github.com/Azure/co-op-translator). Vaikka pyrimme tarkkuuteen, automaattiset käännökset saattavat sisältää virheitä tai epätarkkuuksia. Alkuperäistä asiakirjaa sen alkuperäisellä kielellä tulee pitää ensisijaisena lähteenä. Kriittisissä tapauksissa suositellaan ammattimaista ihmiskääntäjää. Emme ole vastuussa tämän käännöksen käytöstä mahdollisesti aiheutuvista väärinkäsityksistä tai tulkintavirheistä.
